## Queries - Gen3

In this notebook, we show how to query the ComCam repository\
and view the resulting images.\
Craig Lage - 15-Aug-23

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from lsst.daf.butler import Butler
from lsst.ip.isr import IsrTask, IsrTaskConfig

In [ ]:
butler = Butler('/repo/embargo', collections=["LSSTComCamSim/raw/all", "LSSTComCamSim/calib"])

## First, get a list of exposures
### These should match what you see in RubinTV.

In [ ]:
dayObs = 20240425
exposureList = []
for record in butler.registry.queryDimensionRecords("exposure", where="exposure.day_obs=%d"%dayObs):
    exposureList.append([record.id, record])
exposureList.sort(key=lambda x: x[0])
for [id,record] in exposureList:
    print(record.id, record.observation_type, record.exposure_time)


## Next, look at the raw data from one of the exposures.
### Because of the large pedestal, we don't see much.  We need to do ISR

In [ ]:
expId = 2023072800189
raw = butler.get('raw', detector=0, exposure=expId)

In [ ]:
# Now look at the data with matplotlib
from matplotlib.colors import LogNorm

def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

plt.figure(figsize=(8,8))
plt.suptitle(f"Image",fontsize=18)
arr = raw.image.array
img = plt.imshow(arr, norm=LogNorm(vmin=20000, vmax=25000), interpolation='Nearest', cmap='gray')
colorbar(img)
plt.tight_layout(h_pad=1)


## Define a simple ISR
### Just overscan subtraction and bias subtraction.
### This should be enough to look at cosmic rays

In [ ]:
isrConfig = IsrTaskConfig()
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.doAssembleCcd=False
isrConfig.doBias=False
isrConfig.doVariance=False
isrConfig.doLinearize=False
isrConfig.doCrosstalk=False
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=False
isrConfig.doDefect=False
isrConfig.doNanMasking=True
isrConfig.doInterpolate=False
isrConfig.doSaturation=False
isrConfig.doSaturationInterpolation=False
isrTask = IsrTask(config=isrConfig)

## Run the ISR and look at the result
### Here I've taken an exposure where I found a cosmic ray using the RUbinTV CCS image viewer image

In [ ]:
expId = 2023072800189
exp = butler.get('raw', detector=3, exposure=expId) # This is the raw data
#biasExp = butler.get('bias', detector=3, exposure=expId) # This is a bias image associated with the data
isrResult = isrTask.run(exp) # This runs the ISR

In [ ]:
# Now look at the data with matplotlib
# No cosmics are visible at this scale
from matplotlib.colors import LogNorm

def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

plt.figure(figsize=(8,8))
plt.suptitle(f"Image",fontsize=18)
arr = isrResult.exposure.image.array
img = plt.imshow(arr, vmin=-1, vmax=5, interpolation='Nearest', cmap='gray')
colorbar(img)
plt.tight_layout(h_pad=1)


## Now zoom in on a cosmic ray that I found with the RubinTV CCS image viewer

In [ ]:
# Now look at the data with matplotlib
from matplotlib.colors import LogNorm

def colorbar(mappable):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax)
    plt.sca(last_axes)
    return cbar

plt.figure(figsize=(8,8))
plt.suptitle(f"Image",fontsize=18)
arr = isrResult.exposure.image.array[2300:2500,3000:3200] # These coordinates need some explanation
img = plt.imshow(arr, vmin=1, vmax=100, interpolation='Nearest', cmap='gray')
colorbar(img)
plt.tight_layout(h_pad=1)


In [ ]:
test = exp.getDetector()

In [ ]:
amps = test.getAmplifiers()

In [ ]:
amps[0].

In [ ]:
amps[0].getBBox()

In [ ]:
amp1 = exp[amps[0].getRawBBox()]

In [ ]:
amp1.image.array

In [ ]:
amp1Post = isrResult.exposure[amps[0].getBBox()]

In [ ]:
amp1Post.image.array.shape